#LDA topic Model on URdU Datasets
In this notebook i have implemented a NLP technique called Topic Modelling on Urdu News Data using a method called Latent Dirichlet Allocation (LDA).

## Mounting Google Drive
If the dataset is on Google Drive then you have to mount over google drive with collaboratory.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


#Installing required dependencies
First we will install all the required extrnal libraries.  

1.   The pyLDAvis library is used for the visualization part.
2.   lda library is used for training lda model

<b>One thing to remember is that after installing libraries you have to restart the run time again so that other dependencies are not affected by it.</b>

In [ ]:
!pip install lda

In [ ]:
!pip install pyLDAvis


# Importing required dependencies
We will import numpy, pandas and re, lda, gensim library for now. other libraries will be imported in the notebook later.

Pandas will be used to create a Dataframe and handle the csv file. Numpy will be used for the faster computation of arrays to save time. re library will be used for the cleaning of data.

In [ ]:
import lda

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
import pandas as pd
import numpy as np
import re
import gensim
from gensim.models.coherencemodel import CoherenceModel
#optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

# LDA on UNTM Dataset

##DataFrame
We scraped 8001 urdu news that consist of 7 categories=>Business,Technology, showbiz,Health,Politics,weird and Sports. After cleaning news, total 7991 rows saved into another new column Clean_Data for experiments.



In [ ]:
#Load Clean UNTM Dataset from drive
df = pd.read_csv('/content/drive/MyDrive/Clean_UNTM.csv', encoding='utf-8')

In [ ]:
#df.head() is use to examine the top 5 rows of the data by default.
df.head()

,Date,Title,News,Category,Link,Clean_Data
0,2023-04-25,انوشکا شرما اور ویرات کوہلی کی بیڈ منٹن کھیلتے...,ممبئی(اُردو پوائنٹ اخبارتازہ ترین - آن لائن۔ 2...,Sports,https://www.urdupoint.com/sports/detail-news/l...,بھارتی کرکٹ کے آئیکون ویرات کوہلی اور انکی ...
1,2023-04-25,شکست کا ملبہ کسی پر بھی نہیں ڈالوں گا،بابراعظم,راولپنڈی(اُردو پوائنٹ اخبارتازہ ترین - آن لائن...,Sports,https://www.urdupoint.com/sports/detail-news/l...,پاکستان کرکٹ ٹیم کے کپتان بابر اعظم کا کہنا ...
2,2023-04-25,آئی پی ایل:سلواوورریٹ پر کوہلی کو بھاری جرمان...,نئی دہلی (اُردو پوائنٹ اخبارتازہ ترین - آن لائ...,Sports,https://www.urdupoint.com/sports/detail-news/l...,رائل چیلنجرز بنگلور کے کپتان ویرات کوہلی کو ا...
3,2023-04-25,ٹی ٹونٹی ایک مشکل فارمیٹ ہے ،جیت کیلئے اے گیم ...,راولپنڈی (اُردو پوائنٹ اخبارتازہ ترین - آن لائ...,Sports,https://www.urdupoint.com/sports/detail-news/l...,نیوزی لینڈ سے سیریز نہ جیت پانے پر قومی ٹیم ...
4,2023-04-25,آئرلینڈ نے ٹیسٹ کرکٹ میں اپنا سب سے زیادہ ٹیس...,گال(اُردو پوائنٹ اخبارتازہ ترین - آن لائن۔ 25 ...,Sports,https://www.urdupoint.com/sports/detail-news/l...,سری لنکا اور آئرلینڈ کے درمیان جاری دو ٹیسٹ...


## Preprocessing of Data
Data was cleaned that saved into column "Clean_Data" we  removed some extra coloumns first like date,Title,Link and some other columns too. Because we do experiment only news  and all other things are extra for us.

Stopwords are common words that are often filtered out during text processing in natural language processing (NLP) tasks. These words are considered to have little or no value in conveying the actual meaning of the text. We take list of 401 stopwords for topic modelling.

In [ ]:
#drop  columns
df=df.drop(columns=['Date','Title', 'Link'])

In [ ]:
df.head()

,News,Category,Clean_Data
0,ممبئی(اُردو پوائنٹ اخبارتازہ ترین - آن لائن۔ 2...,Sports,بھارتی کرکٹ کے آئیکون ویرات کوہلی اور انکی ...
1,راولپنڈی(اُردو پوائنٹ اخبارتازہ ترین - آن لائن...,Sports,پاکستان کرکٹ ٹیم کے کپتان بابر اعظم کا کہنا ...
2,نئی دہلی (اُردو پوائنٹ اخبارتازہ ترین - آن لائ...,Sports,رائل چیلنجرز بنگلور کے کپتان ویرات کوہلی کو ا...
3,راولپنڈی (اُردو پوائنٹ اخبارتازہ ترین - آن لائ...,Sports,نیوزی لینڈ سے سیریز نہ جیت پانے پر قومی ٹیم ...
4,گال(اُردو پوائنٹ اخبارتازہ ترین - آن لائن۔ 25 ...,Sports,سری لنکا اور آئرلینڈ کے درمیان جاری دو ٹیسٹ...


In [ ]:
documents = df['Clean_Data'].values.tolist()
print((documents[1]))

  پاکستان کرکٹ ٹیم کے کپتان بابر اعظم کا کہنا ہے کہ شکست کا ملبہ کسی پر بھی نہیں ڈالوں گا کیونکہ یہ ٹیم ورک ہوتا ہے اور ہمیشہ ایک ٹیم جیتتی ہے یا ہارتی ہے تاہم جہاں ضرورت ہے اچھا کرنے کی مزید بہتری ضرور لائیں گے۔راولپنڈی میں ٹی  میچز کی سیریز مکمل ہونے کے بعد پاکستان ٹیم کے کپتان بابر اعظم کا کہنا تھا کہ شروع میں اندازہ تھا کہ  سکور کریں گے لیکن  رنز کا اچھا سکور کیا لیکن نیوزی لینڈ کے کھلاڑی اچھا کھیلے۔ میں کبھی مطمئن نہیں ہوتا اور تربیت بھی یہی ہے کہ اچھا کرکے بھی سیکھا جاتا ہے کیونکہ بری پرفارمنس کے بعد تو ہر کوئی سیکھتا ہے۔بابر اعظم کا کہنا تھا کہ دو دن کے وقفے سے کوئی فرق نہیں پڑتا  اوور کے بعد  کا ہدف سمجھ رہے تھے لیکن یہ درست ہے کہ آخری تین اوور میں رضوان سے ہٹ نہیں لگی رضوان مزید دو رنز کر سکتے تھے لیکن نہیں کرسکے بہرحال رضوان نے بہت اچھا کھیلا کیچز ڈراپ ہوئے جو کہ نہیں ہونے چاہیے تھے۔


In [ ]:
#Load stopwords from drive
import nltk
stopwords=pd.read_csv('/content/drive/MyDrive/stopwords.txt',names=['List'])
# stopwords['List']

stopwords_ur=[]
for li in stopwords['List']:
  stopwords_ur.append(li)
print(len(stopwords_ur))
print(stopwords_ur)

401
['کی', 'ہیں', 'ہے', 'رہا', 'رہی', 'رہے', 'تھا', 'تھے', 'تھی', 'تھى', 'میں', 'کہ', 'کے', 'ہوتے', 'کہہ', 'بنانا', 'پھر', 'لکین', 'ہوتی', 'لیتی', 'ایسی', 'ائے', 'ہوئے', 'ہوۓ ', 'مگر', 'چاہے', 'کیے', 'تاکہ', 'تم', 'آکر', 'لگا', 'ہوگیئں', 'ليے', 'رہتی', 'ہوگئی', 'انھوں', 'چاہتے', 'پاگیئں', 'آنا', 'کروا', 'رکھ', 'آتی', 'یہاں', 'جیسا', 'چکے', 'کرئے', 'دیے', 'چکا', 'ملتا', 'کبھی', 'ایسا', 'کرسکیں', 'ہوسکے', 'سکیں', 'لہذا', 'چاہئے', 'وہیں', 'اٹھایا', 'جہنوں', 'ہمارے', 'لیے', 'آرہے', 'کرگیئں', 'بنانے', 'سکتا', 'وغیرہ', 'دے', 'ہوۓ', 'رہنے', 'ہوۓ', 'کئے', 'لگے', 'لگایا', 'لائے', 'کہے', 'کرے', 'رہئں', 'آگئے', 'کئی', 'کم', 'ملی', 'جنہیں', 'ہوئیں', 'تھیں', 'ابھی', 'پاگئیں', 'آئے', 'کرا', 'دیا', 'جہاں', 'آگئیں', 'کرتی', 'رہیں', 'کرتیں', 'دیتی', 'ہوگئے', 'ديتے', 'انہں', 'ایسے', 'رکھتے', 'رہتے', 'رکھی', 'کیں', 'كیا', 'وه', 'جنہيں', 'کروائی', 'دینا', 'جسے', 'جاتی', 'اسکی', 'ملے', 'کرگئى', 'جن', 'آپکو', 'جنہوں', 'دیئے', 'والی', 'جبکہ', 'دیگر', 'آپکا', 'رکھنے', 'انہىں', 'کيے', 'یعنی', 'كيے', 'بننے', 'ج

## Feature Extraction
we used feature extraction technique  countvectorizer used to convert text documents into numerical feature vectors. Then thses feature vectors given to lda for training


In [ ]:
vectorizer = CountVectorizer(stop_words= stopwords_ur)
X = vectorizer.fit_transform(documents)
print(X.toarray())


/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:409: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['او', 'ايم', 'اے'] not in stop_words.
  warnings.warn(


[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


In [ ]:
#Get vocabulary from countvectorizer
vocab=vectorizer.get_feature_names_out()
for i in range(1,200):
  print(vocab[i])

ءبچہ
ءسے
ءمیں
ءکو
ءی
آءی
آؤ
آؤٹ
آؤٹس
آؤڈ
آؤں
آؤے
آئ
آئر
آئرس
آئرش
آئرلینڈ
آئرن
آئرننگ
آئرین
آئزک
آئس
آئسولیشن
آئسولیٹ
آئفی
آئل
آئلنگ
آئلینڈ
آئن
آئند
آئندامر
آئندہ
آئندہے
آئوٹ
آئوں
آئين
آئٹم
آئٹمز
آئیانہوں
آئیتو
آئیحمزہ
آئیر
آئیز
آئیسو
آئیسولیشن
آئیلا
آئین
آئیند
آئیندہ
آئینوں
آئینہ
آئینی
آئینے
آئیوا
آئیووا
آئیوی
آئیٹمز
آئیڈئل
آئیڈنٹیفکیشن
آئیڈیا
آئیڈیاز
آئیڈیل
آئیڈینٹیفکیشن
آئیکمسٹ
آئیکنایوارڈ
آئیکون
آئیگا
آئیگی
آئیں
آئیںایف
آئیے
آئےآئی
آئےاس
آئےجس
آئےضمنی
آئےلوٹا
آئےواضح
آئےگزشتہ
آئےہیںمحمد
آاور
آب
آبا
آباؤ
آبائو
آبائی
آباد
آباداور
آباداورصوابیشامل
آباداورہری
آبادباجوڑتورغرسفرہریپورمانسہرہکولائی
آبادباجوڑتورغرہریپورمانسہرہکولائی
آبادشہروں
آبادضلعی
آبادفروری
آبادفیصل
آبادلائسنس
آبادلیفٹیننٹسہیل
آبادمیڈیکل
آبادمیں
آبادنے
آبادٹوٹلاومیکرون
آبادپراپرٹی
آبادپشاور
آبادپولیس
آبادکا
آبادکاری
آبادکی
آبادکے
آبادہائیکورٹ
آبادی
آبادیوں
آبزرویشن
آبزور
آبشار
آبشارہے
آبشاریں
آبل
آبپارہ
آبپاشی
آبپاشیاں
آبپاشیوں
آبی
آبیاری
آبیانہ
آبیانہتاوان
آتش
آتشبازی
آتشزدگی
آتشی
آتشیں
آتیتفتیش
آتیہر
آتیہیں
آتےمیں
آت

In [ ]:
word2id=vectorizer.vocabulary_
print(word2id)

{'بھارتی': 8556, 'کرکٹ': 38979, 'ئیکون': 809, 'ویرات': 31458, 'کوہلی': 40631, 'انکی': 3768, 'سپر': 18723, 'سٹار': 18494, 'اہلیہ': 5366, 'انوشکا': 3501, 'شرما': 20192, 'بیڈ': 9288, 'منٹن': 27254, 'کھیلتے': 41008, 'ویڈیو': 31651, 'سوشل': 18300, 'میڈیا': 28289, 'وائرل': 30352, 'مطابق': 26501, 'پوما': 34400, 'پروموشنل': 33649, 'ایونٹ': 5925, 'پہنچے': 34974, 'ساتھ': 16952, 'کھیلا': 41000, 'تصاویر': 9866, 'اکثر': 5188, 'فٹنس': 22971, 'وابستگی': 30395, 'روزمرہ': 15390, 'زندگی': 16616, 'کھیلوں': 41014, 'اہمیت': 5370, 'اجاگرکرتے': 1043, 'نظر': 29218, 'تے': 10889, 'مقبولیت': 26774, 'روز': 15367, 'دن': 13719, 'اضافہ': 2062, 'پاکستان': 32920, 'ٹیم': 32443, 'کپتان': 40693, 'بابر': 6383, 'اعظم': 2196, 'شکست': 20518, 'ملبہ': 26892, 'ڈالوں': 36602, 'ورک': 30851, 'ہمیشہ': 44436, 'جیتتی': 12132, 'ہارتی': 43934, 'ضرورت': 21118, 'اچھا': 5065, 'مزید': 26164, 'بہتری': 8924, 'ضرور': 21117, 'لائیں': 23737, 'راولپنڈی': 14851, 'ٹی': 32368, 'میچز': 28254, 'سیریز': 19228, 'مکمل': 27756, 'شروع': 20210, 'اندازہ': 3

## Train LDA with Gibbs Sampling
Latent Dirichlet Allocation (LDA) is a popular probabilistic generative model that based on two main assumptions:


*   Each document in the corpus contains a mixture of different topics.
*   Each topic is represented as a probability distribution over words.

In LDA, Gibbs sampling is used to infer the topic assignments for each word in a document, allowing the model to learn the underlying topic distributions in the corpus.


In [ ]:
#set parameters to train LDA on UNTM(Urdu News Topic Model Dataset)
model = lda.LDA(n_topics=7, n_iter=3500, random_state=15, refresh=20, alpha=0.01,eta=0.01)

In [ ]:
model=model.fit(X)

In [ ]:
#Top 10 words of each topic
n_top_words = 10
topic_word = model.topic_word_
for i, topic_dist in enumerate(topic_word):
    topic_words = np.array(vocab)[np.argsort(topic_dist)][:-(n_top_words+1):-1]
    print('Topic {}: {}'.format(i, ' '.join(topic_words)))

Topic 0: بتایا سال استعمال مطابق وقت ساتھ شخص گھر پہلے لوگوں
Topic 1: ساتھ فلم میڈیا فون صارفین خان اداکارہ سوشل بھارتی اداکار
Topic 2: کورونا وائرس صحت ہزار افراد دوران مریض کیسز لاکھ ہسپتال
Topic 3: روپے سال فیصد ڈالر ملین مالی فی ارب کمی مطابق
Topic 4: پاکستان سی پی ٹی ساتھ فراہم صدر چیمبر اینڈ آف
Topic 5: خان عمران حکومت پی تحریک وزیراعظم لیگ مطابق کوئی پاکستان
Topic 6: ٹیم پاکستان کرکٹ میچ سیریز بال رنز ٹورنامنٹ ٹی لینڈ


# Evaluation
we used three evaluation metrics to compare the results.

1. The coherence score is used to capture the degree of similarity between the words within each topic, with higher scores indicating more coherent topics. We used two coherence metrics NPMI and Cv Score.
2. IRBO measures are used to assess how different and distinct the topics are in a topic model.


### Coherence Score


In [ ]:
from gensim.corpora import Dictionary

In [ ]:
topics_LDA=[]
for index, topic in enumerate(model.components_):
    row=[]
    for i in topic.argsort()[-10:]:
      row.append(vectorizer.get_feature_names_out()[i])
    topics_LDA.append(row)

In [ ]:
print(len(vocab))

46347


In [ ]:
dictionary = Dictionary([vocab])

In [ ]:
print(len(dictionary))

46347


In [ ]:
for i in range(1,200):
  print(dictionary[i])

ءبچہ
ءسے
ءمیں
ءکو
ءی
آءی
آؤ
آؤٹ
آؤٹس
آؤڈ
آؤں
آؤے
آئ
آئر
آئرس
آئرش
آئرلینڈ
آئرن
آئرننگ
آئرین
آئزک
آئس
آئسولیشن
آئسولیٹ
آئفی
آئل
آئلنگ
آئلینڈ
آئن
آئند
آئندامر
آئندہ
آئندہے
آئوٹ
آئوں
آئين
آئٹم
آئٹمز
آئیانہوں
آئیتو
آئیحمزہ
آئیر
آئیز
آئیسو
آئیسولیشن
آئیلا
آئین
آئیند
آئیندہ
آئینوں
آئینہ
آئینی
آئینے
آئیوا
آئیووا
آئیوی
آئیٹمز
آئیڈئل
آئیڈنٹیفکیشن
آئیڈیا
آئیڈیاز
آئیڈیل
آئیڈینٹیفکیشن
آئیکمسٹ
آئیکنایوارڈ
آئیکون
آئیگا
آئیگی
آئیں
آئیںایف
آئیے
آئےآئی
آئےاس
آئےجس
آئےضمنی
آئےلوٹا
آئےواضح
آئےگزشتہ
آئےہیںمحمد
آاور
آب
آبا
آباؤ
آبائو
آبائی
آباد
آباداور
آباداورصوابیشامل
آباداورہری
آبادباجوڑتورغرسفرہریپورمانسہرہکولائی
آبادباجوڑتورغرہریپورمانسہرہکولائی
آبادشہروں
آبادضلعی
آبادفروری
آبادفیصل
آبادلائسنس
آبادلیفٹیننٹسہیل
آبادمیڈیکل
آبادمیں
آبادنے
آبادٹوٹلاومیکرون
آبادپراپرٹی
آبادپشاور
آبادپولیس
آبادکا
آبادکاری
آبادکی
آبادکے
آبادہائیکورٹ
آبادی
آبادیوں
آبزرویشن
آبزور
آبشار
آبشارہے
آبشاریں
آبل
آبپارہ
آبپاشی
آبپاشیاں
آبپاشیوں
آبی
آبیاری
آبیانہ
آبیانہتاوان
آتش
آتشبازی
آتشزدگی
آتشی
آتشیں
آتیتفتیش
آتیہر
آتیہیں
آتےمیں
آت

In [ ]:
texts = [[word for word in str(document).split() if word not in stopwords_ur] for document in documents]

In [ ]:
print(texts[0])

['بھارتی', 'کرکٹ', 'آئیکون', 'ویرات', 'کوہلی', 'انکی', 'سپر', 'سٹار', 'اہلیہ', 'انوشکا', 'شرما', 'بیڈ', 'منٹن', 'کھیلتے', 'ویڈیو', 'سوشل', 'میڈیا', 'وائرل', 'ہوگئی۔بھارتی', 'میڈیا', 'مطابق', 'ویرات', 'کوہلی', 'انوشکا', 'شرما', 'پوما', 'پروموشنل', 'ایونٹ', 'پہنچے', 'ساتھ', 'بیڈ', 'منٹن', 'کھیلا۔جس', 'ویڈیو', 'تصاویر', 'سوشل', 'میڈیا', 'وائرل', 'گئی۔', 'ویرات', 'کوہلی', 'انوشکا', 'شرما', 'اکثر', 'فٹنس', 'وابستگی', 'روزمرہ', 'زندگی', 'کھیلوں', 'اہمیت', 'اجاگرکرتے', 'نظر', 'آتے', 'مقبولیت', 'روز', 'دن', 'اضافہ', 'ہے۔']


In [ ]:
c = CoherenceModel(topics=topics_LDA, texts=texts, dictionary=dictionary , coherence='c_v')
coherence_lda = c.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Coherence Score:  0.624535049266302


### Topic Diversity

In [ ]:
import itertools
from rbo import rbo #upload rbo python file before
import numpy as np

class InvertedRBO:
    def __init__(self):
        pass

    def irbo(self, topics, topk=10, weight=0.9):
        """
        Calculate inverted Rank Biased Overlap (RBO) as a measure of topic diversity from a list of lists of words.

        :param topics: A list of lists of words representing different topics.
        :param topk: The number of top words on which RBO will be computed.
        :param weight: Weight of each agreement at depth d: p**(d-1). When set to 1.0, there is no weight,
                       and the RBO returns to average overlap.
        :return: The inverted RBO topic diversity score.
        """
        if topk <= 0:
            raise ValueError("topk must be a positive integer.")

        num_topics = len(topics)
        if num_topics == 0:
            raise ValueError("topics list cannot be empty.")

        if topk > len(topics[0]):
            raise Exception('Words in topics are less than topk')

        collect = []
        for list1, list2 in itertools.combinations(topics, 2):
            rbo_val = rbo(list1[:topk], list2[:topk], p=weight)[2]
            collect.append(rbo_val)

        Irbo_score = 1 - np.mean(collect)
        return Irbo_score

In [ ]:
inverted_rbo_calculator = InvertedRBO()
IRBO= inverted_rbo_calculator.irbo(topics_LDA, topk=10, weight=0.9)
print("Inverted RBO Score:", IRBO)

Inverted RBO Score: 0.9669632034938775


# LDA on UDC Dataset

##DataFrame
We also performed experiment on publically available dataset for topic modelling that consist of 5 categories=>Business, Entertainment, Health,weird and Sports. After cleaning this news, total 1008 rows saved into another new column Clean_Data for experiments.The link to the dataset is given below:

[Dataset](https://github.com/Mubashar331/Urdu-corpus)




In [ ]:
df1 = pd.read_csv('/content/drive/MyDrive/Clean_BigCorpus.csv', encoding='utf-8')

In [ ]:
df1.head()

,Document,Category,Clean_Data
0,﻿اسلام آباد: ایسوسی ایشن آف چارٹرڈ سرٹیفائی...,Business,اسلام آباد ایسوسی ایشن آف چارٹرڈ سرٹیفائیڈا...
1,اسلام آباد: ایف بی آرنے 1 ارب30کروڑ روپے سے ...,Business,اسلام آباد ایف بی آرنے اربکروڑ روپے سے زائد...
2,کراچی: بزنس مین گروپ کے چیئرمین اور کراچی چیم...,Business,کراچی بزنس مین گروپ کے چیئرمین اور کراچی چیمب...
3,کراچی: ٹرانسپورٹ اور پٹرولیم مصنوعات کی درآم...,Business,کراچی ٹرانسپورٹ اور پٹرولیم مصنوعات کی درآمد...
4,بجٹ کی آمد آمد ہے۔ ایکسپریس میڈیا گروپ کی ی...,Business,بجٹ کی آمد آمد ہے۔ ایکسپریس میڈیا گروپ کی ی...


In [ ]:
documents1 = df1['Clean_Data'].values.tolist()
print((documents1[1]))

 اسلام آباد ایف بی آرنے  اربکروڑ روپے سے زائد مالیت کے گڈز ڈکلیئریشن فنڈ کے استعمال کیلیے خصوصی سیکریٹریٹ قائم کرنے اور ایف بی آر ہیڈ کوارٹرز اورکسٹمز ہاؤس کراچی میں قائم ڈیٹا سینٹرزکی اپ گریڈیشن کرنے کا فیصلہ کیا ہے۔  ایکسپریس کو دستیاب دستاویزکے مطابق یہ اہم فیصلے ایف بی آرکے ممبر کسٹمز زاہدکھوکھرکی زیرصدارت اجلاس میں کیے گئے جس میں یہ بھی فیصلہ کیا گیا کہ کسٹمزکراچی کے وی بوک اوراس کے نیٹ ورک ڈیٹا سینٹر کی فعالیت کو برقرار رکھنے کے لیے پاکستان ریونیو آٹومیشن لمیٹڈ کی جانب سے ضرورت کے مطابق بروقت پروکیورمنٹس کو بھی یقینی بنایا جائے گا تاکہ آئندہ کسٹمز کراچی کے وی بوک نیٹ ورک ڈیٹا سینٹر میں بریک ڈاون کی صورتحال سے بچا جا سکے۔  اجلاس میں یہ بھی فیصلہ کیا گیا کہ اربکروڑ روپے سے زائد مالیت کے گڈز ڈکلیئریشن فنڈ کے استعمال کے لیے خصوصی سیکریٹریٹ قائم کیا جائے گا اور یہ سیکریٹریٹ دسمبر کو جاری کردہ کسٹمز جنرل آرڈر نمبر کے مطابق جی ڈی فنڈ کو استعمال میں لائے گا۔ اس بارے میں ایف بی آر حکام کا کہنا ہے کہ ایف بی آر نے  میں درآمدی کنسائنمنٹس کی پروسیسنگ پر  روپے فی کنسائنمنٹ کے حساب سے 

## Feature Extraction

In [ ]:
vectorizer1 = CountVectorizer(stop_words= stopwords_ur)
X1 = vectorizer1.fit_transform(documents1)
print(X1.toarray())


/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:409: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['او', 'ايم', 'اے'] not in stop_words.
  warnings.warn(


[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


In [ ]:
vocab1=vectorizer1.get_feature_names_out()
for i in range(1,200):
  print(vocab1[i])

آئر
آئرس
آئرلینڈ
آئرن
آئس
آئل
آئندہ
آئین
آئیں
آئیے
آب
آباد
آبادی
آتش
آتیں
آثار
آج
آجائیں
آجائیںگے
آجائے
آجاتی
آخر
آخری
آدمی
آدھ
آدھا
آدھے
آر
آرام
آرتھر
آرم
آرٹسٹ
آرپار
آرچ
آرڈر
آرڈیننس
آرکے
آرہا
آرہی
آزاد
آزاداے
آزادی
آزمائے
آزمانا
آزمایا
آس
آسان
آسانی
آستین
آسمان
آسن
آسٹریلوی
آسٹریلیا
آسٹریلین
آسکتے
آشکار
آصف
آغاز
آف
آفاق
آفریدی
آفس
آفیشل
آفیشلز
آل
آلو
آلوؤں
آلودہ
آم
آماجگاہ
آمادہ
آمد
آمدن
آمدنی
آمیزے
آن
آنتوں
آندرے
آنسوؤں
آنچ
آنکھ
آنکھوں
آنکھیں
آنگن
آواز
آوازآج
آوازاٹھاتے
آور
آویر
آٹا
آٹو
آٹھ
آٹے
آپریشن
آپریٹنگ
آپس
آپشن
آپٹیمس
آڈٹ
آڑو
آڑے
آگ
آگاہ
آگاہی
آگرا
آگہی
آگے
آہستہ
آہنگ
آہنگی
آﺅٹ
ؤنگاجس
ؤٹ
ؤٹس
ؤں
ئر
ئرس
ئرش
ئرلینڈ
ئرن
ئز
ئس
ئسو
ئش
ئل
ئن
ئند
ئندہ
ئوبھگت
ئوٹ
ئٹر
ئٹم
ئٹمزفیصدمہنگے
ئٹمزپر
ئی
ئین
ئینوجوان
ئینگےیہ
ئینہ
ئینی
ئیڈیا
ئیڈیل
ئیڈیلائز
ئیکن
ئیکون
ئیگین
ئیں
ئیںگے
ئیے
ئے
ئےا
ئےاور
ئےدوسرے
اؤنڈر
ائل
ائیر
ائیرپسٹل
ابارشن
ابال
ابالیں
ابتدا
ابتداء
ابتدائی
ابتدامیں
ابتر
ابدی
ابراج
ابرار
ابرام
ابراہم
ابراہیم
ابلاغ
ابلتی
ابلتے
ابلے
ابن
ابنارمل
ابنارمیلٹی
ابو
ابواب
ابوظبی
ابوظہبی
ابو

In [ ]:
word2id1=vectorizer1.vocabulary_
print(word2id1)

{'اسلام': 538, 'آباد': 12, 'ایسوسی': 2272, 'ایشن': 2286, 'چارٹرڈ': 15937, 'سرٹیفائیڈاکاؤئنٹنٹس': 7875, 'ٹیکس': 14565, 'نظام': 13266, 'سنجیدگی': 8133, 'جانچ': 4890, 'پڑتال': 15450, 'ضرورت': 9320, 'دہندگان': 6454, 'عوام': 9701, 'حکام': 5596, 'اعتماد': 845, 'انتہائی': 1260, 'نتیجے': 13166, 'چوری': 16125, 'باعث': 2665, 'قومی': 10586, 'خزانہ': 5766, 'اربوں': 427, 'روپے': 7055, 'نقصان': 13315, 'اے': 2529, 'سی': 8550, 'جانب': 4869, 'جمع': 5060, 'کرائی': 17196, 'تجاویز': 4070, 'شرح': 8889, 'یک': 20203, 'ہندسی': 19617, 'ڈائریکٹ': 16429, 'ٹیکسیشن': 14575, 'رفتار': 6881, 'اضافے': 811, 'نادرا': 13025, 'ڈیٹا': 16823, 'بیس': 3829, 'ودہولڈنگ': 13840, 'استعمال': 513, 'اثاثہ': 246, 'جات': 4811, 'تشخیص': 4220, 'شامل': 8804, 'اسٹرکچرل': 600, 'ریفارمز': 7283, 'دہندہ': 6455, 'تمام': 4350, 'امور': 1195, 'کیلیے': 18526, 'ریٹرن': 7335, 'تجویز': 4088, 'کیاگیا': 18440, 'کہاگیا': 18369, 'پاکستان': 14733, 'ٹو': 14371, 'جی': 5359, 'ڈی': 16737, 'پی': 15712, 'خطے': 5795, 'کارپوریٹ': 16985, 'فیصد': 10343, 'بلند': 319

## Train LDA

In [ ]:
model1 = lda.LDA(n_topics=5, n_iter=3500, random_state=15, refresh=20)

In [ ]:
model1=model1.fit(X1)

In [ ]:
n_top_words = 10
topic_word = model1.topic_word_
for i, topic_dist in enumerate(topic_word):
    topic_words = np.array(vocab1)[np.argsort(topic_dist)][:-(n_top_words+1):-1]
    print('Topic {}: {}'.format(i, ' '.join(topic_words)))

Topic 0: ٹیم پاکستان کیلیے سی کرکٹ ساتھ محمد علی کراچی ٹیسٹ
Topic 1: فلم خان ساتھ بھارتی ووڈ اداکارہ میڈیا بالی سلمان اداکار
Topic 2: استعمال کینسر پانی صحت علاج ماہرین خواتین ساتھ جسم افراد
Topic 3: روپے پاکستان حکومت سال فیصد ٹیکس ارب ڈالر بجٹ جانب
Topic 4: کوئی گھر دنیا وقت بات سال لوگوں خاتون لوگ نام


In [ ]:
doc_topic = model1.transform(X1)
predicted_labels1 = np.argmax(doc_topic, axis=1)
print(predicted_labels1)

[3 3 3 ... 4 2 4]


## Evaluation

**Coherence Score**

In [ ]:
topics_LDA1=[]
for index, topic in enumerate(model1.components_):
    row=[]
    for i in topic.argsort()[-10:]:
      row.append(vectorizer1.get_feature_names_out()[i])
    topics_LDA1.append(row)

In [ ]:
print(topics_LDA1)

[['ٹیسٹ', 'کراچی', 'علی', 'محمد', 'ساتھ', 'کرکٹ', 'سی', 'کیلیے', 'پاکستان', 'ٹیم'], ['اداکار', 'سلمان', 'بالی', 'میڈیا', 'اداکارہ', 'ووڈ', 'بھارتی', 'ساتھ', 'خان', 'فلم'], ['افراد', 'جسم', 'ساتھ', 'خواتین', 'ماہرین', 'علاج', 'صحت', 'پانی', 'کینسر', 'استعمال'], ['جانب', 'بجٹ', 'ڈالر', 'ارب', 'ٹیکس', 'فیصد', 'سال', 'حکومت', 'پاکستان', 'روپے'], ['نام', 'لوگ', 'خاتون', 'لوگوں', 'سال', 'بات', 'وقت', 'دنیا', 'گھر', 'کوئی']]


In [ ]:
dictionary1 = Dictionary([vocab1])

In [ ]:
for i in range(1,200):
  print(dictionary1[i])

آئر
آئرس
آئرلینڈ
آئرن
آئس
آئل
آئندہ
آئین
آئیں
آئیے
آب
آباد
آبادی
آتش
آتیں
آثار
آج
آجائیں
آجائیںگے
آجائے
آجاتی
آخر
آخری
آدمی
آدھ
آدھا
آدھے
آر
آرام
آرتھر
آرم
آرٹسٹ
آرپار
آرچ
آرڈر
آرڈیننس
آرکے
آرہا
آرہی
آزاد
آزاداے
آزادی
آزمائے
آزمانا
آزمایا
آس
آسان
آسانی
آستین
آسمان
آسن
آسٹریلوی
آسٹریلیا
آسٹریلین
آسکتے
آشکار
آصف
آغاز
آف
آفاق
آفریدی
آفس
آفیشل
آفیشلز
آل
آلو
آلوؤں
آلودہ
آم
آماجگاہ
آمادہ
آمد
آمدن
آمدنی
آمیزے
آن
آنتوں
آندرے
آنسوؤں
آنچ
آنکھ
آنکھوں
آنکھیں
آنگن
آواز
آوازآج
آوازاٹھاتے
آور
آویر
آٹا
آٹو
آٹھ
آٹے
آپریشن
آپریٹنگ
آپس
آپشن
آپٹیمس
آڈٹ
آڑو
آڑے
آگ
آگاہ
آگاہی
آگرا
آگہی
آگے
آہستہ
آہنگ
آہنگی
آﺅٹ
ؤنگاجس
ؤٹ
ؤٹس
ؤں
ئر
ئرس
ئرش
ئرلینڈ
ئرن
ئز
ئس
ئسو
ئش
ئل
ئن
ئند
ئندہ
ئوبھگت
ئوٹ
ئٹر
ئٹم
ئٹمزفیصدمہنگے
ئٹمزپر
ئی
ئین
ئینوجوان
ئینگےیہ
ئینہ
ئینی
ئیڈیا
ئیڈیل
ئیڈیلائز
ئیکن
ئیکون
ئیگین
ئیں
ئیںگے
ئیے
ئے
ئےا
ئےاور
ئےدوسرے
اؤنڈر
ائل
ائیر
ائیرپسٹل
ابارشن
ابال
ابالیں
ابتدا
ابتداء
ابتدائی
ابتدامیں
ابتر
ابدی
ابراج
ابرار
ابرام
ابراہم
ابراہیم
ابلاغ
ابلتی
ابلتے
ابلے
ابن
ابنارمل
ابنارمیلٹی
ابو
ابواب
ابوظبی
ابوظہبی
ابو

In [ ]:
texts1 = [[word for word in str(document).split() if word not in stopwords_ur] for document in documents1]

In [ ]:
print(texts1[0])

['اسلام', 'آباد', 'ایسوسی', 'ایشن', 'آف', 'چارٹرڈ', 'سرٹیفائیڈاکاؤئنٹنٹس', 'ٹیکس', 'نظام', 'سنجیدگی', 'جانچ', 'پڑتال', 'ضرورت', 'ٹیکس', 'دہندگان', 'عوام', 'ٹیکس', 'حکام', 'اعتماد', 'انتہائی', 'نتیجے', 'ٹیکس', 'چوری', 'باعث', 'قومی', 'خزانہ', 'اربوں', 'روپے', 'نقصان', 'اے', 'سی', 'سی', 'اے', 'جانب', 'جمع', 'کرائی', 'ٹیکس', 'تجاویز', 'ٹیکس', 'شرح', 'یک', 'ہندسی', 'ڈائریکٹ', 'ٹیکسیشن', 'رفتار', 'اضافے', 'نادرا', 'ڈیٹا', 'بیس', 'ودہولڈنگ', 'ٹیکس', 'ڈیٹا', 'استعمال', 'اثاثہ', 'جات', 'تشخیص', 'شامل', 'ہیں۔', 'اے', 'سی', 'سی', 'اے', 'اسٹرکچرل', 'ریفارمز', 'ٹیکس', 'دہندہ', 'تمام', 'ٹیکس', 'امور', 'کیلیے', 'ٹیکس', 'ریٹرن', 'تجویز', 'کیاگیا', 'تجاویز', 'کہاگیا', 'پاکستان', 'ٹیکس', 'ٹو', 'جی', 'ڈی', 'پی', 'شرح', 'خطے', 'کارپوریٹ', 'ٹیکس', 'شرح', 'فیصد', 'بلند', 'سطح', 'ٹیکس', 'حکام', 'ضرورت', 'سیلز', 'ٹیکس', 'خطے', 'انتہائی', 'بلند', 'فیصد', 'ایشیا', 'اوسطاً', 'فیصد', 'قریب', 'سیلز', 'ٹیکس', 'براہ', 'راست', 'ٹیکس', 'متبادل', 'بجائے', 'ٹیکس', 'بیس', 'توسیع', 'کیلیے', 'استعمال', 'چاہیے۔']


In [ ]:
c1 = CoherenceModel(topics=topics_LDA1, texts=texts1, dictionary=dictionary1 , coherence='c_npmi')
coherence_lda1 = c1.get_coherence()
print('\nCoherence Score: ', coherence_lda1)


Coherence Score:  0.09177982566326098


**Diversity Score**

In [ ]:
print("topic_diversity:",topic_diversity(topics_LDA1, topk=10))

topic_diversity: 0.92


In [ ]:
inverted_rbo_calculator = InvertedRBO()
IRBO= inverted_rbo_calculator.irbo(topics_LDA1, topk=10, weight=0.9)
print("Inverted RBO Score:", IRBO)

Inverted RBO Score: 0.9720312536928571
